In [4]:
from com_1_edit_distance import com_1_edit_distance
import string
import pandas as pd

In [63]:
sentence = "மளை பெய்தால், விவசாயிகளுக்கு அதிக வருமானம் கிடைக்கும்"

In [64]:
tokens = sentence.split(" ")

In [65]:
tokens

['மளை', 'பெய்தால்,', 'விவசாயிகளுக்கு', 'அதிக', 'வருமானம்', 'கிடைக்கும்']

In [66]:
_tokens = tokens.copy()
for i in range(len(tokens)):
    _token = list(tokens[i])
    __token = _token.copy()
    for unit in _token:
        if unit in string.punctuation:
            __token.remove(unit)
    
    _tokens[i] = "".join(__token)
_tokens

['மளை', 'பெய்தால்', 'விவசாயிகளுக்கு', 'அதிக', 'வருமானம்', 'கிடைக்கும்']

In [67]:
import pandas as pd

In [68]:
words_with_root = pd.read_csv("./src/MostusedRootwords.csv", header=None)
words_with_root.head()

,0,1,2
0,Root,NaN,derivation
1,செய்,889.0,"செய்து,செய்ய,செய்யவும்,செய்வதில்,செய்யும்,செய்..."
2,இது,868.0,"இது,இதை,இதுகுறித்து,இதனை,இதையும்,இதனைக்,இத்தகை..."
3,இரு,851.0,"இருந்தால்,இருக்கும்,இருக்கும்பொழுது,இருந்து,இர..."
4,மேல்,805.0,"மேலும்,மேல்,மேலே,மேலிலிருந்து,மேல்குறித்த,மேலு..."


In [1]:
non_words = []
for token in _tokens:
    if not words_with_root[2].str.contains(token).any():
        non_words.append(token)

NameError: name '_tokens' is not defined

In [70]:
non_words

['மளை']

In [71]:
wiki = pd.read_csv("./src/cleaned_wiktionary_words.csv", header=None)
wiki.head()

,0
0,ஃ
1,ஃபாசிசம்
2,ஃபைலேரியா
3,அ
4,அஃக


In [72]:
channel_probs = {}
for non_word in non_words:
    print("Word:", non_word)
    for i in range(len(wiki)):
        res = com_1_edit_distance(wiki.iloc[i, 0], non_word)        
        if res is not None:
            key = wiki.iloc[i, 0] + "^" + non_word + "^" + str(res)
            channel_probs[key] = 0
            print(key)

Word: மளை
அமளை^மளை^(1, 'deletion', 0, None)
அளை^மளை^(1, 'substitution', 0, None)
ஆளை^மளை^(1, 'substitution', 0, None)
இளை^மளை^(1, 'substitution', 0, None)
ஈளை^மளை^(1, 'substitution', 0, None)
உளை^மளை^(1, 'substitution', 0, None)
ஊளை^மளை^(1, 'substitution', 0, None)
களை^மளை^(1, 'substitution', 0, None)
தளை^மளை^(1, 'substitution', 0, None)
நளை^மளை^(1, 'substitution', 0, None)
பளை^மளை^(1, 'substitution', 0, None)
மடை^மளை^(1, 'substitution', 1, None)
மணை^மளை^(1, 'substitution', 1, None)
மதளை^மளை^(1, 'deletion', 1, None)
மனை^மளை^(1, 'substitution', 1, None)
மரை^மளை^(1, 'substitution', 1, None)
மறை^மளை^(1, 'substitution', 1, None)
மலை^மளை^(1, 'substitution', 1, None)
மழை^மளை^(1, 'substitution', 1, None)
மாளை^மளை^(1, 'deletion', 1, None)
மிளை^மளை^(1, 'deletion', 1, None)
முளை^மளை^(1, 'deletion', 1, None)
மூளை^மளை^(1, 'deletion', 1, None)
மை^மளை^(1, 'insertion', 1, None)
வளை^மளை^(1, 'substitution', 0, None)


In [73]:
import numpy as np
import pickle

all_letters_unicode_b10 = np.array([2947,2949,2950,2951,2952,2953,2954,2958,2959,2960,2962,2963,2964,2965,2969,2970,2972,2974,2975,2979,2980,2984,2985,2986,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000,3001,3006,3007,3008,3009,3010,3014,3015,3016,3018,3019,3020,3021])
with open("./src/_count.pkl", "rb") as file:
    _count = pickle.load(file)

count_array = np.load("./src/count_array.npy")
sub = np.load("./src/sub.npy")
tra = np.load("./src/tra.npy")
ins = np.load("./src/ins.npy")
_del = np.load("./src/del.npy")

def compute_channel_probability(x, y, err):
    if err[1] == "insertion":
        i = np.where(all_letters_unicode_b10 == ord(y[err[2]-1]))[0][0]
        j = np.where(all_letters_unicode_b10 == ord(y[err[2]]))[0][0]
        
        return ins[i, j] / count_array[i]

    elif err[1] == "transposition":
        i = np.where(all_letters_unicode_b10 == ord(x[err[2]]))[0][0]
        j = np.where(all_letters_unicode_b10 == ord(x[err[3]]))[0][0]
        
        return tra[i, j] / _count[(i, j)]
    
    elif err[1] == "substitution":
        pre = np.where(all_letters_unicode_b10 == ord(x[err[2]]))[0][0]
        post = np.where(all_letters_unicode_b10 == ord(y[err[2]]))[0][0]

        print(sub[pre, post], count_array[post])

        return sub[pre, post] / count_array[post]
        
    
    elif err[1] == "deletion":
        i = np.where(all_letters_unicode_b10 == ord(x[err[2]-1]))[0][0]
        j = np.where(all_letters_unicode_b10 == ord(x[err[2]]))[0][0]
        
        return _del[i, j] / _count[(i, j)]
    
    else:
        raise Exception("Unkown Error Cause")

for key in channel_probs:
    x, y, err = key.split("^")
    err = eval(err)
    channel_probs[key] = compute_channel_probability(x, y, err)

1.0 48359.0
1.0 48359.0
1.0 48359.0
1.0 48359.0
1.0 48359.0
1.0 48359.0
1.0 48359.0
1.0 48359.0
1.0 48359.0
1.0 48359.0
1.0 10486.0
1.0 10486.0
1.0 10486.0
1.0 10486.0
1.0 10486.0
7947.0 10486.0
1381.0 10486.0
1.0 48359.0


In [74]:
channel_probs

{"அமளை^மளை^(1, 'deletion', 0, None)": 0.3333333333333333,
 "அளை^மளை^(1, 'substitution', 0, None)": 2.067867408341777e-05,
 "ஆளை^மளை^(1, 'substitution', 0, None)": 2.067867408341777e-05,
 "இளை^மளை^(1, 'substitution', 0, None)": 2.067867408341777e-05,
 "ஈளை^மளை^(1, 'substitution', 0, None)": 2.067867408341777e-05,
 "உளை^மளை^(1, 'substitution', 0, None)": 2.067867408341777e-05,
 "ஊளை^மளை^(1, 'substitution', 0, None)": 2.067867408341777e-05,
 "களை^மளை^(1, 'substitution', 0, None)": 2.067867408341777e-05,
 "தளை^மளை^(1, 'substitution', 0, None)": 2.067867408341777e-05,
 "நளை^மளை^(1, 'substitution', 0, None)": 2.067867408341777e-05,
 "பளை^மளை^(1, 'substitution', 0, None)": 2.067867408341777e-05,
 "மடை^மளை^(1, 'substitution', 1, None)": 9.536524890329964e-05,
 "மணை^மளை^(1, 'substitution', 1, None)": 9.536524890329964e-05,
 "மதளை^மளை^(1, 'deletion', 1, None)": 0.004915615271178109,
 "மனை^மளை^(1, 'substitution', 1, None)": 9.536524890329964e-05,
 "மரை^மளை^(1, 'substitution', 1, None)": 9.5365248

In [76]:
sorted(channel_probs.values(), reverse=True)

[0.7578676330345222,
 0.3333333333333333,
 0.2626959247648903,
 0.1316994087354568,
 0.12881658123467796,
 0.06052883083784645,
 0.02008295132067234,
 0.004915615271178109,
 9.536524890329964e-05,
 9.536524890329964e-05,
 9.536524890329964e-05,
 9.536524890329964e-05,
 9.536524890329964e-05,
 2.067867408341777e-05,
 2.067867408341777e-05,
 2.067867408341777e-05,
 2.067867408341777e-05,
 2.067867408341777e-05,
 2.067867408341777e-05,
 2.067867408341777e-05,
 2.067867408341777e-05,
 2.067867408341777e-05,
 2.067867408341777e-05,
 2.067867408341777e-05,
 2.067867408341777e-05]